In [1]:
!pip install pyLDAvis
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 14.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=396729abad35f377cc5431396920e5a9c1384faebec484846d54cdb1643b14bc
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=a9f5f7742be3ed47678017e3d581110bd3c91fe154cbfc6aebc66044deaddab4
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


# Prepare Stopwords

In [3]:
import nltk
nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load Dataset (for only Donald Trump)

In [5]:
# LoadDataset
df=pd.read_excel('speeches.xlsx')
df=df.drop('info',axis=1)
df.head()
#list1 = ['Joe Biden','Barack Obama','George W. Bush','Bill Clinton']
list1 = ['Donald Trump']
#for i in (list1):

df = df[df['name'].isin(list1)]
df

,name,title,date,speech
17,Donald Trump,"January 19, 2021: Farewell Address",2021-01-19,"My fellow Americans: Four years ago, we lau..."
18,Donald Trump,"January 13, 2021: Statement about the Violence...",2021-01-13,"My fellow Americans, I want to speak to yo..."
19,Donald Trump,"January 7, 2021: Message After Pro-Trump Mob O...",2021-01-07,I would like to begin by addressing the hei...
20,Donald Trump,"January 6, 2021: Speech Urging Supporters to G...",2021-01-06,I know your pain. I know your hurt. We ha...
21,Donald Trump,"November 5, 2020: Remarks on the 2020 Election",2020-11-05,THE PRESIDENT: Good evening. I’d like to p...
22,Donald Trump,"October 26, 2020: Swearing-In Ceremony of the ...",2020-10-26,THE PRESIDENT: Thank you very much. Ap...
23,Donald Trump,"September 26, 2020: Announcing His Nominee for...",2020-09-26,THE PRESIDENT: Thank you very much. Thank ...
24,Donald Trump,"September 7, 2020: Labor Day Press Conference",2020-09-07,THE PRESIDENT: Thank you very much. Thank ...
25,Donald Trump,"August 8, 2020: Press Conference on Executive ...",2020-08-08,"Thank you very much everybody. Thank you, ..."
26,Donald Trump,"July 4, 2020: Remarks at Salute to America",2020-07-04,THE PRESIDENT: Wow. Are you having a good ...


# Remove newline and other characters

In [6]:
# Convert to list 
data = df.speech.values.tolist()  

# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

[' My fellow Americans: Four years ago, we launched a great national effort to '
 'rebuild our country, to renew its spirit, and to restore the allegiance of '
 'this government to its citizens. In short, we embarked on a mission to make '
 'America great again — for all Americans. As I conclude my term as the 45th '
 'President of the United States, I stand before you truly proud of what we '
 'have achieved together. We did what we came here to do — and so much more. '
 'This week, we inaugurate a new administration and pray for its success in '
 'keeping America safe and prosperous. We extend our best wishes, and we also '
 'want them to have luck — a very important word. I’d like to begin by '
 'thanking just a few of the amazing people who made our remarkable journey '
 'possible. First, let me express my overwhelming gratitude for the love and '
 'support of our spectacular First Lady, Melania. Let me also share my deepest '
 'appreciation to my daughter Ivanka, my son-in-law Jar

<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<>:5: DeprecationWarning: invalid escape sequence \s
<ipython-input-6-6b7da4b5f6f0>:5: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


# Tokenize words and cleanup the text

In [7]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['my', 'fellow', 'americans', 'four', 'years', 'ago', 'we', 'launched', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'to', 'renew', 'its', 'spirit', 'and', 'to', 'restore', 'the', 'allegiance', 'of', 'this', 'government', 'to', 'its', 'citizens', 'in', 'short', 'we', 'embarked', 'on', 'mission', 'to', 'make', 'america', 'great', 'again', 'for', 'all', 'americans', 'as', 'conclude', 'my', 'term', 'as', 'the', 'th', 'president', 'of', 'the', 'united', 'states', 'stand', 'before', 'you', 'truly', 'proud', 'of', 'what', 'we', 'have', 'achieved', 'together', 'we', 'did', 'what', 'we', 'came', 'here', 'to', 'do', 'and', 'so', 'much', 'more', 'this', 'week', 'we', 'inaugurate', 'new', 'administration', 'and', 'pray', 'for', 'its', 'success', 'in', 'keeping', 'america', 'safe', 'and', 'prosperous', 'we', 'extend', 'our', 'best', 'wishes', 'and', 'we', 'also', 'want', 'them', 'to', 'have', 'luck', 'very', 'important', 'word', 'like', 'to', 'begin', 'by', 'thanking', 'just'

# Creating Bigram and Trigram models

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['my_fellow_americans', 'four', 'years', 'ago', 'we', 'launched', 'great', 'national', 'effort', 'to', 'rebuild', 'our', 'country', 'to', 'renew', 'its', 'spirit', 'and', 'to', 'restore', 'the', 'allegiance', 'of', 'this', 'government', 'to', 'its', 'citizens', 'in', 'short', 'we', 'embarked', 'on', 'mission', 'to', 'make', 'america', 'great', 'again', 'for', 'all', 'americans', 'as', 'conclude', 'my', 'term', 'as', 'the', 'th', 'president', 'of', 'the', 'united_states', 'stand', 'before', 'you', 'truly', 'proud', 'of', 'what', 'we', 'have', 'achieved', 'together', 'we', 'did', 'what', 'we', 'came', 'here', 'to', 'do', 'and', 'so', 'much', 'more', 'this', 'week', 'we', 'inaugurate', 'new', 'administration', 'and', 'pray', 'for', 'its', 'success', 'in', 'keeping', 'america', 'safe', 'and', 'prosperous', 'we', 'extend', 'our', 'best', 'wishes', 'and', 'we', 'also', 'want', 'them', 'to', 'have', 'luck', 'very', 'important', 'word', 'like', 'to', 'begin', 'by', 'thanking', 'just', 'few', '

# Remove Stopwords, make bigrams and lemmatize

In [9]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Installing SpaCy

In [10]:
#Installing SpaCy
!conda install -c conda-forge spacy
!python -m spacy download en_core_web_lg

/bin/bash: conda: command not found
2022-10-18 02:18:43.070051: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 587.7 MB 10 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


# Call the functions in order

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['fellow_american', 'year', 'ago', 'launch', 'great', 'national', 'effort', 'rebuild', 'country', 'renew', 'restore', 'allegiance', 'government', 'citizen', 'short', 'embark', 'mission', 'make', 'conclude', 'term', 'th', 'stand', 'truly', 'proud', 'achieve', 'together', 'come', 'much', 'week', 'new', 'administration', 'pray', 'success', 'keep', 'safe', 'prosperous', 'extend', 'good', 'wish', 'also', 'want', 'luck', 'important', 'word', 'begin', 'thank', 'amazing', 'people', 'make', 'remarkable', 'journey', 'possible', 'first', 'let', 'express', 'overwhelming', 'gratitude', 'love', 'support', 'spectacular', 'let', 'also', 'share', 'deep', 'appreciation', 'daughter', 'law', 'jar', 'fill', 'light', 'joy', 'also', 'want', 'wonderful', 'wife', 'entire', 'family', 'thank', 'well', 'chief', 'staff', 'mark', 'meadow', 'dedicate', 'member', 'white_house', 'staff', 'cabinet', 'incredible', 'people', 'administration', 'pour', 'heart', 'also', 'want', 'take', 'moment', 'thank', 'truly', 'exceptio

# Create Dictionary and Corpus needed for Topic Modeling

In [13]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 2), (1, 1), (2, 2), (3, 1), (4, 7), (5, 1), (6, 1), (7, 5), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 2), (15, 1), (16, 8), (17, 6), (18, 1), (19, 17), (20, 2), (21, 1), (22, 1), (23, 1), (24, 2), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 4), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 3), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 3), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 2), (54, 1), (55, 5), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 3), (67, 1), (68, 1), (69, 2), (70, 1), (71, 4), (72, 1), (73, 3), (74, 10), (75, 1), (76, 1), (77, 1), (78, 1), (79, 7), (80, 1), (81, 3), (82, 2), (83, 1), (84, 1), (85, 3), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 4), (92, 1), (93, 1), (94, 1), (95, 16), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 4), (107, 2), (108, 1), (109, 1), (110,

# Building topic model

# After some trial and error, we thought that nop. of topics = 10 gave a decent split.

In [14]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics = 10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

[(0,
  '0.019*"go" + 0.017*"wall" + 0.014*"deal" + 0.014*"drug" + 0.013*"want" + '
  '0.012*"lot" + 0.012*"get" + 0.010*"much" + 0.009*"people" + 0.009*"dollar"'),
 (1,
  '0.011*"people" + 0.010*"country" + 0.010*"american" + 0.008*"great" + '
  '0.008*"want" + 0.008*"thank" + 0.007*"nation" + 0.007*"go" + 0.007*"year" + '
  '0.007*"work"'),
 (2,
  '0.022*"applause" + 0.018*"thank" + 0.012*"go" + 0.011*"people" + '
  '0.011*"great" + 0.010*"life" + 0.010*"know" + 0.009*"much" + 0.008*"say" + '
  '0.008*"love"'),
 (3,
  '0.008*"people" + 0.007*"world" + 0.006*"kill" + 0.006*"terrorist" + '
  '0.005*"good" + 0.005*"american" + 0.005*"thank" + 0.005*"take" + '
  '0.005*"military" + 0.004*"great"'),
 (4,
  '0.001*"go" + 0.000*"people" + 0.000*"say" + 0.000*"year" + 0.000*"thank" + '
  '0.000*"country" + 0.000*"nation" + 0.000*"want" + 0.000*"great" + '
  '0.000*"applause"'),
 (5,
  '0.017*"go" + 0.016*"virus" + 0.011*"state" + 0.009*"get" + 0.007*"say" + '
  '0.007*"see" + 0.007*"look" + 0

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -7.286077612986705

Coherence Score:  0.29797052513953803


# Visualizing the topics

In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.029268  0.034209       1        1  31.608684
6      0.163019 -0.046279       2        1  23.422623
9      0.083631  0.045657       3        1  23.054269
2      0.073466  0.070273       4        1   6.282936
8      0.074430 -0.026726       5        1   6.112694
7     -0.052270  0.153421       6        1   4.189782
5     -0.026828 -0.117883       7        1   3.057842
0     -0.019372 -0.100898       8        1   1.720509
3     -0.142011  0.007139       9        1   0.549209
4     -0.183333 -0.018913      10        1   0.001452, topic_info=          Term        Freq       Total Category  logprob  loglift
202         go  843.000000  843.000000  Default  30.0000  30.0000
199        get  571.000000  571.000000  Default  29.0000  29.0000
2301  applause  483.000000  483.000000  Default  28.0000  28.0000
455        say  667.000000  667.000000  Default  27.0000  27.0000
367     people  824.000000  824.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
60        call    0.000262  205.801904  Topic10  -8.0928  -2.4327
177      first    0.000256  168.992220  Topic10  -8.1160  -2.2588
337      never    0.000258  235.297545  Topic10  -8.1083  -2.5822
79        come    0.000263  320.860373  Topic10  -8.0905  -2.8745
461        see    0.000258  319.177551  Topic10  -8.1082  -2.8869

[802 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
3674      3  0.847187  accomplishment
2559      7  0.847606          acting
599       1  0.645683          action
599       2  0.061494          action
599       3  0.092240          action
...     ...       ...             ...
595       5  0.041577            year
595       6  0.025408            year
595       8  0.004620            year
595       9  0.002310            year
4092      8  0.716584             zte

[1655 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 7, 10, 3, 9, 8, 6, 1, 4, 5])

# Repeating the entire process for Bill Clinton, George W. Bush, Barack Obama and Joe Biden

In [20]:
# LoadDataset
df=pd.read_excel('speeches.xlsx')
df=df.drop('info',axis=1)
df.head()
list1 = ['Joe Biden','Barack Obama','George W. Bush','Bill Clinton']
#for i in (list1):

df = df[df['name'].isin(list1)]

# Convert to list 
data = df.speech.values.tolist()  

# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  



def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))


# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example


# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 

#Running LDA
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics = 10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

<>:14: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<>:14: DeprecationWarning: invalid escape sequence \s
<ipython-input-20-e315abd1a754>:14: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7

[(0,
  '0.000*"applause" + 0.000*"year" + 0.000*"people" + 0.000*"nation" + '
  '0.000*"go" + 0.000*"time" + 0.000*"work" + 0.000*"get" + 0.000*"world" + '
  '0.000*"come"'),
 (1,
  '0.020*"cut" + 0.012*"make" + 0.012*"government" + 0.010*"american" + '
  '0.010*"people" + 0.008*"work" + 0.008*"go" + 0.007*"time" + 0.007*"tax" + '
  '0.007*"family"'),
 (2,
  '0.017*"people" + 0.011*"war" + 0.009*"peace" + 0.008*"world" + '
  '0.007*"country" + 0.007*"force" + 0.006*"ally" + 0.006*"right" + '
  '0.005*"leader" + 0.005*"conflict"'),
 (3,
  '0.000*"make" + 0.000*"people" + 0.000*"nation" + 0.000*"go" + 0.000*"work" '
  '+ 0.000*"know" + 0.000*"year" + 0.000*"country" + 0.000*"take" + '
  '0.000*"come"'),
 (4,
  '0.021*"people" + 0.019*"go" + 0.015*"get" + 0.015*"say" + 0.013*"think" + '
  '0.009*"know" + 0.009*"work" + 0.009*"want" + 0.008*"well" + 0.008*"make"'),
 (5,
  '0.010*"people" + 0.010*"world" + 0.010*"nation" + 0.007*"peace" + '
  '0.006*"make" + 0.006*"war" + 0.006*"life" + 0.0

In [21]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -7.433997087776062

Coherence Score:  0.31641946602720444


In [22]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.119510  0.115685       1        1  28.004408
9      0.167406 -0.006121       2        1  27.097689
7      0.177698 -0.070806       3        1  24.420825
4      0.160669 -0.022151       4        1  13.855521
2      0.043075  0.149667       5        1   3.684633
1      0.042105 -0.163117       6        1   1.687822
6     -0.074213  0.022570       7        1   0.985814
8     -0.223023 -0.007357       8        1   0.261442
3     -0.206569 -0.009173       9        1   0.000933
0     -0.206658 -0.009197      10        1   0.000913, topic_info=       Term         Freq        Total Category  logprob  loglift
408   right   664.000000   664.000000  Default  30.0000  30.0000
347  people  2530.000000  2530.000000  Default  29.0000  29.0000
742     cut   383.000000   383.000000  Default  28.0000  28.0000
292    make  1563.000000  1563.000000  Default  27.0000  27.0000
499    want   819.000000   819.000000  Default  26.0000  26.0000
..      ...          ...          ...      ...      ...      ...
423     see     0.000207   634.250875  Topic10  -9.0732  -3.3324
463   thank     0.000207   602.266782  Topic10  -9.0733  -3.2807
564    help     0.000207   958.307269  Topic10  -9.0735  -3.7453
277    life     0.000207   640.576742  Topic10  -9.0736  -3.3426
208    good     0.000207   766.247794  Topic10  -9.0738  -3.5221

[810 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
8253      4  0.197255       accuse
8253      7  0.690391       accuse
2356      4  0.854586         ache
2610      1  0.461400  achievement
2610      2  0.188755  achievement
...     ...       ...          ...
1103      2  0.611979        young
1103      3  0.073732        young
1103      4  0.092166        young
1103      5  0.040553        young
1103      7  0.011060        young

[1524 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 10, 8, 5, 3, 2, 7, 9, 4, 1])